In [1]:
from utils import *
from tracker import CentroidTracker

import os
import cv2 as cv
import numpy as np

/Users/daryashitova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
'''loading video names into a list for easier access with absolute path,
video names can be access by calling the index of the video_lst and will contain 
the current absolute path, asumes that the video folder is in the same folder as this file'''

video_path = "../videos"
full_path = os.path.abspath(video_path)

video_lst = []
v_dir = os.listdir(full_path)
for name in v_dir:
    video_lst.append(full_path+'/'+name)
    
video_lst = sorted(video_lst)

In [4]:
'''Frame differencing code based on:
https://github.com/infoaryan/OPENCV-PYTHON-Zero-to-One-Course-Resources/blob/master/Video%2031%20-%20Frame%20Differencing/frame_differencing.py
Expanded with: blur, dilation and morphological noise reduction and contour finding

Threshold for noise reduction and contour finding is adapted dynamically and will be 
calculated in every frame
'''

#video to capture
video = sorted(video_lst)[0]

kernel = np.ones((7,7)) #kerne for dilation, erosion
kernel_blur = (3,3)
kernel_morph = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7)) #kernel for morphology
threshold = 100 #when to detect difference
VALUE = 255 #which value to assign to difference
frame_counter = 0
power = 1 #magnifying factor for adaptive threshold

#threshold to calculate
threshold_method = cv.THRESH_BINARY

#settting the tracker 
tracker = CentroidTracker()
centerpoints = []
tracks = {}

cap =cv.VideoCapture(video)
prev_frame = get_frame(cap)
cur_frame = get_frame(cap)
next_frame = get_frame(cap)


'''Check if directory for outputs exists, if not, create it'''

# You should change 'test' to your preferred folder.
dir = ("outputs")
check = os.path.isdir(dir)
# If folder doesn't exist, then create it.
if not check:
    os.makedirs(dir)

file_name = video.split("/")[-1].split(".")
file_name[0] = file_name[0]+"_out"
output_name = ".".join(file_name)
full_output_path = f"{dir}/{output_name}"

#write for output videos
result = cv.VideoWriter(full_output_path, cv.VideoWriter_fourcc(*'XVID'), 20, (640, 512))

# Iterating over all frames and applying difference and dilation
while True:
    frames = [prev_frame,cur_frame,next_frame]
    frame_counter += 1
    #applying preprocessing
    prev_frame, cur_frame, next_frame = prepare_frames(frames, kernel_blur)
    
    #calculating difference
    frame_difference = frame_diff(prev_frame, cur_frame, next_frame)
   
    #applying dilation
    frame_difference = cv.erode(frame_difference, (1,1))
    frame_difference = cv.dilate(frame_difference, kernel)    
    #applying morphological noise reduction
    frame_difference = cv.morphologyEx(frame_difference, cv.MORPH_OPEN, kernel_morph)   
    #grayscale conversion
    frame_difference = cv.cvtColor(frame_difference, cv.COLOR_BGR2GRAY)

    #applying thresholds
    ret, frame_th = cv.threshold(frame_difference, threshold, VALUE, threshold_method)

    #finding and drawing contours on given frame with given threshold
    detections = construct_contours(frame_th, threshold*2)
    
    frame_difference_blured = cv.blur(frame_difference,(3,3)) # needed for adapting threshold
    
    '''Dynamic threshold adaption'''
    threshold_new = np.quantile(frame_difference_blured, 0.9998)
    frame_skip = 1#how many frames to skip
    
    if frame_counter == 1:
        threshold = threshold_new
    if frame_counter % frame_skip == 0: 
        if threshold_new > 0.995**power * threshold:
            threshold = threshold_new
            power = 1
        if threshold_new <= 0.995**power * threshold:
            power += 1
        
    '''Tracker'''
    #showing the track only after 10th frame to avoid false positives in the adaptation stage
    if frame_counter > 10:
        #getting ID and centroid for each of the detections
        boxes_ids = tracker.update(detections)
        for (objectID, centroid) in boxes_ids.items():
            if objectID in tracks:
                tracks[objectID].append(centroid)
            else:
                tracks[objectID] = [centroid]
            
            centerpoints.append((centroid[0], centroid[1]))
            text = f"ID {objectID}"
            #plotting ID and bounding boxes if the track is longer than 40 frames to avoid false positives plotting
            if len(tracks[objectID]) > 40:
                cv.putText(cur_frame, text, (centroid[0], centroid[1]-15),
                    cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0, 255, 0), 2)
                cv.rectangle(cur_frame, (centroid[0]-10, centroid[1]-10), (centroid[0]+10, centroid[1]+10), (0,255,0), 2)

        #plotting the track if it is longer than 40 frames to avoid false positives
        for key in tracks:
            if len(tracks[key]) > 40:
                for i, point in enumerate(tracks[key][1:]):
                    cv.line(cur_frame, tracks[key][i], point, (0,255,0), 3) 
            
    result.write(cur_frame)
    # Update the variables
    prev_frame = cur_frame
    cur_frame = next_frame
    # next_frame = get_frame(cap)
    ret, next_frame = cap.read()

    if not ret:
        break

    if cv.waitKey(25) & 0xFF == ord('q'):
        break    
cap.release()
result.release()
cv.waitKey(1)

cv.destroyAllWindows()
for i in range (1,5):
    cv.waitKey(1)

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
